<a href="https://colab.research.google.com/github/nkcong206/Travel-Recommendation-System/blob/main/crawl_data/query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import psycopg2

In [43]:
!pip install -q langchain
!pip install -q langchain_community
!pip install -q langchain-ollama

In [ ]:
!pip install -qU langchain-google-genai

In [ ]:
!pip install geopy

In [73]:
import requests
from bs4 import BeautifulSoup as bp
import time
import csv
import json
from geopy.geocoders import Nominatim
import re

In [ ]:
conn = psycopg2.connect(database = "travellocations",
                        user = "postgres",
                        host= 'localhost',
                        password = "cong2006",
                        port = 5432)

In [ ]:
cur = conn.cursor()

# Thực thi các lệnh SQL để tạo schema, loại dữ liệu, và bảng
cur.execute("""
DROP SCHEMA IF EXISTS places CASCADE;
CREATE SCHEMA IF NOT EXISTS places;
SET search_path TO places;

CREATE TYPE address AS (
    street TEXT,
    district TEXT,
    city TEXT
);

CREATE TYPE location AS (
    latitude DECIMAL(9, 6),
    longitude DECIMAL(9, 6)
);

CREATE TABLE places.hotels (
    hotel_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    address address,
    location location,
    rating DECIMAL(2, 1),
    description TEXT,
    img_url JSON,
    comments TEXT
);

CREATE TABLE places.hotel_price_range (
    id SERIAL PRIMARY KEY,
    hotel_id INT REFERENCES places.hotels(hotel_id) ON DELETE CASCADE,
    room_type VARCHAR(100),
    occupancy INT,
    price DECIMAL(10, 2)
);
""")

# Cam kết các thay đổi vào cơ sở dữ liệu
conn.commit()

# # Đóng con trỏ và kết nối
cur.close()
# conn.close()


In [44]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
REQUEST_HEADER = {
    'User-Agent': USER_AGENT,
    'Accept-language': 'en-US, en;q=0.5',
}

In [ ]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [45]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 12030    0 12030    0     0  43071      0 --:--:-- --:--:-- --:--:-- 42964
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [46]:
import subprocess
import time
import threading

# Start the ollama server in a new process
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Function to print server output
def print_output(process):
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.strip().decode('utf-8'))
        time.sleep(1)

# Start a thread to print server output
thread = threading.Thread(target=print_output, args=(process,))
thread.start()

print("Ollama server is running in the background")

Ollama server is running in the background


In [ ]:
#!ollama serve&

In [37]:
!ollama pull llama3:latest

[GIN] 2024/08/18 - 05:22:56 | 200 |      29.284µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏  29 KB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏ 8.5 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  31 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  62 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   2% ▕▏  70 MB/4.7 GB                  pul

In [47]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3:latest", temperature=0)

In [79]:
def get_page_html(url):
    res = requests.get(url=url, headers=REQUEST_HEADER)
    return res.text

def get_hotel_price(soup):
    price_element = soup.find('div', attrs={'style': 'color: rgb(255, 94, 31); font-size: 20px;'})
    if price_element:
        true_price = price_element.text.strip().replace('VND', '').replace('.', '')
        return float(true_price)
    return None

def get_hotel_name(soup):
    name = soup.find('div', class_='css-901oao r-a5wbuh r-1enofrn r-b88u0q r-1cwl3u0 r-fdjqy7 r-3s2u2q')
    return name.text.strip() if name else None

def get_hotel_rating(soup):
    rating = soup.find('div', class_='css-901oao r-jwli3a r-a5wbuh r-s67bdx r-b88u0q r-10cxs7j r-q4m81j')
    return rating.text.strip() if rating else None

def get_hotel_des(soup):
    des = soup.find('div', attrs={'style': 'font-family:Godwit, -apple-system, BlinkMacSystemFont, Segoe UI, Roboto, Arial, sans-serif, Apple Color Emoji, Segoe UI Emoji, Segoe UI Symbol;font-size:14px;line-height:20px;max-height:80px;overflow:hidden'})
    return des.text.strip().replace('\n', '') if des else None

def get_hotel_address(soup):
    address = soup.find('div', class_='css-901oao css-cens5h r-13awgt0 r-a5wbuh r-1b43r93 r-majxgm r-rjixqe r-fdjqy7')
    if address:
        address_full = address.text.strip().replace('\t', '')

        prompt = f"""
            Please separate the following address into 3 parts: street, district, city:
            {address_full}

            Ensure that the "district" value is one of the following 12 options: Ba Đình, Cầu Giấy, Đống Đa, Hai Bà Trưng, Hoàn Kiếm, Thanh Xuân, Hoàng Mai, Long Biên, Hà Đông, Tây Hồ, Nam Từ Liêm, Bắc Từ Liêm.
            The "city" value must be Hà Nội.

            Provide the result in the following JSON format:
            {{
            "street": "...",
            "district": "...",
            "city": "..."
            }}
        """
        response = llm.invoke(prompt)
        response_text = str(response.content)
        cleaned_json_str = re.search(r'\{.*?\}', response_text, re.DOTALL).group(0)
        try:
            result_dict = json.loads(cleaned_json_str)
            return result_dict
        except json.JSONDecodeError:
            return {'street': '', 'district': '', 'city': ''}

def get_hotel_location(address):
    addr = f"{address['street']}, {address['district']}, {address['city']}"
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(addr)

    if location:
        latitude = location.latitude
        longitude = location.longitude
        return {'latitude': latitude, 'longitude': longitude}
    else:
        return {'latitude': '', 'longitude': ''}

def get_hotel_comments(soup):
    comments = []
    a = soup.findAll('div', class_='css-901oao css-cens5h r-cwxd7f r-a5wbuh r-1b43r93 r-majxgm r-rjixqe r-fdjqy7')
    for comment in a:
        comments.append(comment.text.strip())
    return comments

def insert_hotel_data(conn, info):
    cur = conn.cursor()
    try:
        cur.execute("""
            INSERT INTO places.hotels (name, address, location, rating, description, img_url, comments)
            VALUES (%s, ROW(%s, %s, %s), ROW(%s, %s), %s, %s, %s, %s)
            RETURNING hotel_id;
        """, (
            info['name'],
            info['address']['street'],
            info['address']['district'],
            info['address']['city'],
            info['location']['latitude'],
            info['location']['longitude'],
            info['rating'],
            info['description'],
            info.get('img_url', None),
            info['comments']
        ))

        hotel_id = cur.fetchone()[0]

        if 'price' in info:
            cur.execute("""
                INSERT INTO places.hotel_price_range (hotel_id, room_type, occupancy, price)
                VALUES (%s, %s, %s, %s)
            """, (
                hotel_id,
                info.get('room_type', None),
                info.get('occupancy', None),
                info['price']
            ))

        conn.commit()
    except Exception as e:
        print(f"Error inserting data: {e}")
        conn.rollback()
    finally:
        cur.close()

def extract_hotels_info(url):
    info = {}
    html = get_page_html(url)
    soup = bp(html, 'lxml')
    info['name'] = get_hotel_name(soup)
    info['price'] = get_hotel_price(soup)
    info['rating'] = get_hotel_rating(soup)
    info['address'] = get_hotel_address(soup)
    info['location'] = get_hotel_location(info['address'])
    info['description'] = get_hotel_des(soup)
    info['comments'] = get_hotel_comments(soup)
    return info


In [ ]:
db_params = {
    'database': 'travellocations',
    'user': 'postgres',
    'host': 'localhost',
    'password': 'cong2006',
    'port': 5432
}

with open('hotels.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        url = row[0]
        data = extract_hotels_info(url)
        insert_hotel_data(conn, data)
        time.sleep(0.25)

# conn.close()

In [80]:
url = "https://www.traveloka.com/vi-vn/hotel/vietnam/hanoi-sunshine-hotel-1000000468475?spec=18-08-2024.19-08-2024.1.1.HOTEL.1000000468475..2"
html = get_page_html(url)
soup = bp(html, 'lxml')
address = get_hotel_address(soup)
print(address)
location = get_hotel_location(address)
print(location)

[GIN] 2024/08/18 - 07:52:41 | 200 |  2.340038235s |       127.0.0.1 | POST     "/api/chat"
{'street': '18 Hàng Hòm, Hàng Gai', 'district': 'Hoàn Kiếm', 'city': 'Hà Nội'}
{'latitude': 21.031777, 'longitude': 105.8488738}


In [77]:
url = "https://www.traveloka.com/vi-vn/hotel/vietnam/hanoi-sunshine-hotel-1000000468475?spec=18-08-2024.19-08-2024.1.1.HOTEL.1000000468475..2"
html = get_page_html(url)
soup = bp(html, 'lxml')
address = soup.find('div', class_='css-901oao css-cens5h r-13awgt0 r-a5wbuh r-1b43r93 r-majxgm r-rjixqe r-fdjqy7')
address_full = address.text.strip().replace('\t', '')
print(address_full)
prompt = f"""
        Please separate the following address into 5 parts: street, district , and city:
        {address_full}

        Ensure that the "district" value is one of the following 12 options: Ba Đình, Cầu Giấy, Đống Đa, Hai Bà Trưng, Hoàn Kiếm, Thanh Xuân, Hoàng Mai, Long Biên, Hà Đông, Tây Hồ, Nam Từ Liêm, Bắc Từ Liêm.
        The "city" value must be Hà Nội.

        Provide the result in the following JSON format:
        {{
        "street": "...",
        "district": "...",
        "city": "..."
        }}
    """
import re
response = llm.invoke(prompt)
response_text = str(response.content)
print(response_text)
json_str = re.search(r'\{.*?\}', response_text, re.DOTALL).group(0)
print(json_str)
result_dict = json.loads(json_str)
result_dict

18 Hàng Hòm, Hàng Gai, Quận Hoàn Kiếm, Hà Nội, Việt Nam, 100000
INFO [main] build info | build=1 commit="1e6f655" tid="138658424987648" timestamp=1723967429
INFO [main] system info | n_threads=1 n_threads_batch=-1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="138658424987648" timestamp=1723967429 total_threads=2
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="40065" tid="138658424987648" timestamp=1723967429
INFO [main] model loaded | tid="138658424987648" timestamp=1723967433
[GIN] 2024/08/18 - 07:50:35 | 200 |  6.169984996s |       127.0.0.1 | POST     "/api/chat"
Here is the separated address in the requested JSON format:

{
"street": "18 Hàng Hòm, Hàng Gai",
"district": "Hoàn Kiếm",
"city": "Hà Nội"
}

Note tha

{'street': '18 Hàng Hòm, Hàng Gai', 'district': 'Hoàn Kiếm', 'city': 'Hà Nội'}

In [56]:
from geopy.geocoders import Nominatim

def geocode_address_osm(address):
    # Initialize the geocoder
    geolocator = Nominatim(user_agent="my_geocoder")

    # Geocode the address
    location = geolocator.geocode(address)

    if location:
        return {
            'latitude': location.latitude,
            'longitude': location.longitude,
            'address': location.address
        }
    else:
        return None

# Example usage
address = "hàng hòm, hàng gai, Hoàn Kiếm, Hà Nội"
result = geocode_address_osm(address)

if result:
    print(f"Latitude: {result['latitude']}, Longitude: {result['longitude']}")
    print(f"Address: {result['address']}")
else:
    print("Address not found.")


Latitude: 21.031777, Longitude: 105.8488738
Address: Phố Hàng Hòm, Phường Hàng Gai, Khu phố cổ, Quận Hoàn Kiếm, Thành phố Hà Nội, 11015, Việt Nam


In [91]:
# Thông tin user-agent để giả lập trình duyệt
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
REQUEST_HEADER = {
    'User-Agent': USER_AGENT,
    'Accept-language': 'en-US, en;q=0.5',
}

def get_page_html(url):
    res = requests.get(url=url, headers=REQUEST_HEADER)
    return res.text

def get_restaurant_name(soup):
    name = soup.find('h1', itemprop='name')
    return name.text.strip() if name else None

def get_restaurant_address(soup):
    # Extract address components
    street_address = soup.find('span', itemprop='streetAddress')
    address_locality = soup.find('span', itemprop='addressLocality')
    address_region = soup.find('span', itemprop='addressRegion')

    address_data = {
        'street': street_address.get_text(strip=True) if street_address else None,
        'district': address_locality.get_text(strip=True) if address_locality else None,
        'city': address_region.get_text(strip=True) if address_region else None
    }
    return address_data

def get_restaurant_location(address):
    addr = f"{address['street']}, {address['district']}, {address['city']}"
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(addr)

    if location:
        latitude = location.latitude
        longitude = location.longitude
        return {'latitude': latitude, 'longitude': longitude}

def get_restaurant_rating(soup):
    rating = soup.find('div', itemprop='ratingValue', class_='microsite-point-avg')
    return rating.get_text(strip=True) if rating else None

def get_restaurant_description(soup):
    description = soup.find('span', itemprop='ratingValue')
    return description.text.strip() if description else None


def get_restaurant_comments(soup):
    comments = []
    comment_elements = soup.findAll('div', class_='comment')
    for comment in comment_elements:
        comments.append(comment.text.strip())
    return comments

def extract_restaurant_info(url):
    info = {}
    html = get_page_html(url=url)
    soup = bp(html, 'lxml')
    info['name'] = get_restaurant_name(soup)
    info['address'] = get_restaurant_address(soup)
    info['location'] = get_restaurant_location(info['address'])
    info['rating'] = get_restaurant_rating(soup)
    info['description'] = get_restaurant_description(soup)
    info['comments'] = get_restaurant_comments(soup)
    return info


In [92]:
url = "https://www.foody.vn/ha-noi/kfc-tay-son"
html = get_page_html(url)
soup = bp(html, 'lxml')
print(get_restaurant_name(soup))
print(get_restaurant_rating(soup))

Gà Rán KFC - Tây Sơn
6.7
